In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import distributions as dist

from metropolis import acceptance_rate_per_step, conditional_prob, mala_step, lagk_ac

# General Setup

We have a target distribution and we will try to sample from it using Metropolis-Hastings MCMC with MALA proposal technqiues using various proposal distributions.

Some useful links were found here:  
https://en.wikipedia.org/wiki/Metropolis-adjusted_Langevin_algorithm  
http://www.mcmchandbook.net/HandbookChapter1.pdf  
https://theclevermachine.wordpress.com/2012/11/19/a-gentle-introduction-to-markov-chain-monte-carlo-mcmc/  

Information on lag-k autocorrelation:
http://www.itl.nist.gov/div898/handbook/eda/section3/eda35c.htm

Given measurements, $Y_1, Y_2, ..., Y_N$ at time $X_1, X_2, ..., X_N$, the lag k autocorrelation function is defined as
\begin{equation}
    r_k = \frac{\sum_{i=1}^{N-k}(Y_{i} - \bar{Y})(Y_{i+k} - \bar{Y})}{\sum_{i=1}^{N}(Y_{i} - \bar{Y})^{2}}
\end{equation}

In [ ]:
# Select bimodal
if False:
    mean1 = [2, 0]
    cov1 = [[1, 0], [0, 1]]

    mean2 = [-2, 0]
    cov2 = [[1, 0], [0, 1]]

    tau = 1.0
    
    rv = dist.bimodal_dist(mean1, mean2, cov1, cov2)
    
# Select parabolic gaussian
if True:
    mean = [0, 0]
    cov = [[1, 0], [0, 1]]
    
    tau = 0.4

    rv = dist.parabolic_gaussian(mean, cov, warp=0.9)
    
# Select streched gaussian
if False:
    mean = [0, 0]
    cov = [[1, 0], [0, 100]]
    
    tau = 1.0
    
    rv = dist.stretched_normal(mean, cov)

In [ ]:
# Samples
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.plot(*rv.rvs(1000).T, marker='o', linestyle='None')
ax1.axis('equal')
plt.show()

x, y = np.mgrid[-10:10:.1,
                -10:10:.1]
pos = np.dstack((x, y))

# pdf
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(x, y, rv.pdf(pos))
plt.show()

# logpdf
fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
ax3.contourf(x, y, rv.logpdf(pos))
plt.show()

In [ ]:
# magnitude of gradpdf
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(x, y, np.linalg.norm(rv.grad_pdf(pos), axis=0))
plt.show()

# magnitude of grad log pdf
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(x, y, np.linalg.norm(rv.grad_logpdf(pos), axis=0))
plt.show()

## MALA Proposal

We propose by suggesting the state plus the gradient of the target distribution at the state plus a small random motion.

Note, for reference, the derivate with respect to the state of our target distribution is given by:
\begin{equation}
    \frac{\partial \pi(\mathbf{x})}{\partial\mathbf{x}}=-\frac{1}{\sqrt{det(2\pi\mathbf{\Sigma})}}exp\left[-\frac{1}{2}(\mathbf{x}-\mathbf{m})^{T}\mathbf{\Sigma}^{-1}(\mathbf{x}-\mathbf{m})\right]\mathbf{\Sigma}^{-1}(\mathbf{x}-\mathbf{m})
\end{equation}

However, this is a bit irrelevant since I am looking for the gradient at the state. I.e. $\nabla\pi(\textbf{x})$ I will do it numerically.

In [ ]:
# MCMC Execution
states = [np.array([0,0])]
steps = 1000

for i in range(steps):
    states.append(mala_step(states[-1], rv, v=False, log=True, tau=tau))
    
states = np.asarray(states)

In [ ]:
# Acceptance rate plot
acps = acceptance_rate_per_step(states)
plt.figure(figsize=(8, 4))
plt.plot(range(steps), acps)
plt.xlabel('Steps')
plt.ylabel('Acceptance Rate')
plt.show()

In [ ]:
# Sample and trace plots
df = pd.DataFrame(states, columns=["x", "y"])

plt.figure(1, figsize=(8,8))
plt.subplot(121)
plt.scatter(df.x, df.y, c=rv.pdf(states))
plt.axis('equal')
plt.title('Samples plot')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(222)
plt.plot(np.arange(states.shape[0]), df.x)
plt.title('Trace plot')
plt.xlabel('Steps')
plt.ylabel('x')

plt.subplot(224)
plt.plot(np.arange(states.shape[0]), df.y)
plt.xlabel('Steps')
plt.ylabel('y')

plt.show()

In [ ]:
# def lag_line(sequence, k):
#     plot_ac = []
#     variance = np.sum(np.dot(sequence - np.mean(sequence, axis=0), (sequence - np.mean(sequence, axis=0)).T).diagonal())

#     for i in range(len(sequence)):
#         if i < k:
#             pass
#         else:
#             plot_ac.append([i, lagk_ac(sequence[:i], k, v=False, variance=variance)])
#     return np.asarray(plot_ac)

# lag1 = lag_line(states, 1)
# lag5 = lag_line(states, 5)
# lag10 = lag_line(states, 10)
# lag50 = lag_line(states, 50)

# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.plot(lag1[:,0], lag1[:,1])
# ax.plot(lag5[:,0], lag5[:,1])
# ax.plot(lag10[:,0], lag10[:,1])
# ax.plot(lag50[:,0], lag50[:,1])
# plt.show()

In [ ]:
def autocorr(x):
    """
    Computes the ( normalised) auto-correlation function of a
    one dimensional sequence of numbers.
    
    Utilises the numpy correlate function that is based on an efficient
    convolution implementation.
    
    Inputs:
    x - one dimensional numpy array
    
    Outputs:
    Vector of autocorrelation values for a lag from zero to max possible
    """
    
    # normalise, compute norm
    xunbiased = x - np.mean(x)
    xnorm = np.sum(xunbiased ** 2)
    
    # convolve with itself
    acor = np.correlate(xunbiased, xunbiased, mode='same')
    
    # use only second half, normalise
    acor = acor[int(len(acor) / 2):] / xnorm
    
    return acor


y = autocorr(states[:,0])

plt.plot(y)
plt.show()